Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


# LAB9

Write a local-search algorithm (eg. an EA) able to solve the _Problem_ instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

- Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
- Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

- Reviews will be assigned on Monday, December 4
- You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)


In [2]:
import random
from tqdm import tqdm
import lab9_lib

In [3]:
# fitness = lab9_lib.make_problem(1)
# for n in range(10):
#     ind = random.choices([0, 1], k=10)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)

In [13]:
class LocalSearchAlgorithm:
    def __init__(self, problem_instance, genome_length, population_size=50, generations=1000):
        self.problem_instance = problem_instance
        self.genome_length = genome_length
        self.population_size = population_size
        self.generations = generations

    def initialize_population(self):
        return [random.choices([0, 1], k=self.genome_length) for _ in range(self.population_size)]

    def evaluate_population(self, population):
        return [(individual, self.problem_instance(individual)) for individual in population]

    def select_parents(self, evaluated_population, num_parents=2):
        return [individual for individual, _ in sorted(evaluated_population, key=lambda x: x[1], reverse=True)[:num_parents]]

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, self.genome_length - 1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        return child

    def mutate(self, individual, mutation_rate=0.01):
        return [bit if random.random() > mutation_rate else 1 - bit for bit in individual]

    def run(self):
        population = self.initialize_population()

        for generation in range(self.generations):
            evaluated_population = self.evaluate_population(population)

            for _ in range(self.population_size // 2):
                parents = self.select_parents(evaluated_population)
                child = self.crossover(*parents)
                child = self.mutate(child)
                population.append(child)

            population = population[:self.population_size]

        best_individual, best_fitness = max(evaluated_population, key=lambda x: x[1])
        return best_individual, best_fitness


genome_length = 1000

problem_1 = lab9_lib.make_problem(1)
problem_2 = lab9_lib.make_problem(2)
problem_5 = lab9_lib.make_problem(5)
problem_10 = lab9_lib.make_problem(10)

algorithm_1 = LocalSearchAlgorithm(problem_1, genome_length)
algorithm_2 = LocalSearchAlgorithm(problem_2, genome_length)
algorithm_5 = LocalSearchAlgorithm(problem_5, genome_length)
algorithm_10 = LocalSearchAlgorithm(problem_10, genome_length)


# Parallel thanks chatGPT
import concurrent.futures

# Create a list of algorithms
algorithms = [algorithm_1, algorithm_2, algorithm_5, algorithm_10]

# Create a function to run an algorithm and return the result
def run_algorithm(algorithm):
    return algorithm.run()

# Create a ThreadPoolExecutor with the desired number of threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the tasks to the executor and store the futures
    futures = [executor.submit(run_algorithm, algorithm) for algorithm in algorithms]

    # Wait for all the tasks to complete and get the results
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for i, result in enumerate(results):
    print(f"Problem {i+1}: {result[1]:.2}")




Problem 1: 0.2
Problem 2: 0.51
Problem 3: 0.53
Problem 4: 0.11
